In [3]:
def dynamic_tsfresh ():

    #Changing Work Folder
    
    add_path1 = "/Input/"
    add_path2 = "/.Kernel/"
    base_path = os.getcwd()
    working_path = os.getcwd()
    Input_path = working_path + add_path1
    Kernel_path = working_path + add_path2
    
    # Change folder to Kernel
    
    os.chdir( Kernel_path )

    # Load the the filtered features from the seed data-set

    features_filtered = pd.read_csv('features_filtered_3.csv')

    # Extract the useful information of it

    columns = np.array(features_filtered.columns)
    kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(features_filtered.columns)

    sensors_names = [None] * int(features_filtered.shape[1]);


    for i in range (columns.shape[0]):
        name = columns[i]
        c = '__';
        words = name.split(c)

        sensors_names[i] = words[0]

        '''if i < 20:

            print(name)
            print(words)
            print(features_names[i])
            print(sensors_names[i])
            print('_______')'''

    columns = columns.tolist()
    unique_sensors_names = np.unique(np.array(sensors_names))

    # Change folder to Input

    os.chdir( Input_path )

    # Load the incoming data
    
    Data = np.genfromtxt('Output_3.csv', delimiter=',')
    data_frame = pd.DataFrame(Data[:,0:11], columns= ['id','time'] + ['Sensor_' + str(x) for x in range(1,(Data.shape[1]-2))])

    # Feature extraction guided by the seed data-set

    extraction_df = pd.DataFrame(data_frame.loc[::,'id':unique_sensors_names[0]].values,columns= ['id','time','Sensor'])
    #print(extraction_df.head())
    arrayList = [] 

    for sensor in unique_sensors_names:
        
        #print(extraction_df.head())
        #print('_____')
        extraction_df.loc[::,'Sensor'] = data_frame.loc[::,sensor]
        
        #print(extraction_df.head())
        #print('_____')
        
        extraction_df = extraction_df.rename(columns={'Sensor': sensor})
        
        tsfresh_parameters = kind_to_fc_parameters[sensor]
        
        extracted_features = extract_features(extraction_df, column_id="id", column_sort="time", default_fc_parameters=tsfresh_parameters)

        arrayList.append(extracted_features)

        extraction_df = extraction_df.rename(columns={sensor : 'Sensor'})    

    original_space_features = pd.concat(arrayList,axis=1)

    # Sort the features in accordance with the seed data-set
    
    original_space_features = original_space_features[columns]
    impute(original_space_features)
    original_space_features.sort_index(inplace = True)

    # Change folder to origin
    
    os.chdir( base_path )
    
    return original_space_features

In [4]:
def PCA_projection (features):
    
    #Changing Work Folder
        
    add_path3 = "/.Kernel/"
  
    base_path = os.getcwd()
    working_path = os.getcwd()
    Kernel_path = working_path + add_path3
        
    # Now change to PCA Figures directory

    os.chdir( Kernel_path )

    # load the model from disk
    loaded_pca = pickle.load(open('pca.sav', 'rb'))

    scaler = StandardScaler().fit(features)
    features_padronizadas = scaler.transform(features)

    features_reduzidas = loaded_pca.transform(features_padronizadas)
    
    #print('Filtered Features')
    #print('-' * 20)
    #print(np.size(features_padronizadas,0))
    #print(np.size(features_padronizadas,1))
    #print('-' * 20)
    #print('Reduced Features')
    #print('-' * 20)
    #print(np.size(features_reduzidas,0))
    #print(np.size(features_reduzidas,1))
    
    # Now chance to base directory
    
    os.chdir( base_path )
    
    return features_reduzidas

In [5]:
def Model_Predict (projected_data):
    
    add_path2 = "/.Kernel/";
    add_path3 = "/.Recovery/";
    base_path = os.path.dirname(os.path.abspath("Model_Unified_Code.ipynb"));
    Kernel_path = base_path + add_path2;
    
    # Now change to Kernel directory
    
    os.chdir( Kernel_path )
    
    model = pickle.load(open('model.sav', 'rb'))
    
    for i in range (projected_data.shape[0]):
        
        y_predict = model.predict(projected_data[i,:].reshape(1, -1))
    
        if y_predict[0] == 0:
            print('Ferramenta Boa')
        else:
            print('Ferramenta Ruim')
    
        #print ('Label de Teste: %d' % int (projected_data[i]))
        print ('Label dado pale NN: %d' % int (y_predict[0]))
        print('___________________')
        print('                   ')
        
    # Now change to the base directory
    
    os.chdir( base_path )

In [27]:
import os
import pandas as pd
import numpy as np
import tsfresh 
from tsfresh import extract_features
import pickle
from tsfresh.utilities.dataframe_functions import impute
from sklearn.preprocessing import StandardScaler


In [28]:
os.chdir('/home/thiago/Repositories/Lathes_Tool_Project/RaspberryPi/IPython/')

In [29]:
features = dynamic_tsfresh()

projected_data = PCA_projection(features)

Model_Predict(projected_data)

Feature Extraction: 100%|██████████| 10/10 [00:02<00:00,  3.70it/s]


Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
                   
Ferramenta Ruim
Label dado pale NN: 1
___________________
      

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator PCA from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [36]:
import glob
import os

os.chdir('/home/thiago/Repositories/Lathes_Tool_Project/RaspberryPi/IPython/Input/')

i = 0;

all_files = glob.glob('/home/thiago/Repositories/Lathes_Tool_Project/RaspberryPi/IPython/.Kernel/*.csv') #give path to your desired file path
latest_csv = max(all_files, key=os.path.getctime)
data = np.genfromtxt(latest_csv,delimiter=',')  
print (data)

os.chdir('/home/thiago/Repositories/Lathes_Tool_Project/RaspberryPi/IPython/')

[1. 3. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 3.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 3. 3. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.